In [1]:
from qiskit_aer import AerSimulator
from qiskit import *
import supermarq
from qiskit.providers.fake_provider import *
import numpy as np 
import os
import matplotlib.pyplot as plt 
########## INPUT ############
f=5 # number of fake_backend want to choose
lower_limit= 4 # the lower limit of qubits for the circuit
upper_limit= 5 # the lower limit of qubits for the circuit
steps= 1 #jump in qubit numbers
name_of_algo='vqe'

In [2]:
#all_fakes=[FakeAlmadenV2,FakeArmonkV2,FakeAthensV2,FakeAuckland,FakeBelemV2,FakeBoeblingenV2,FakeBogotaV2,FakeBrooklynV2,FakeBurlingtonV2,FakeCairoV2,FakeCambridgeV2,FakeCasablancaV2,FakeEssexV2,FakeGeneva,FakeGuadalupeV2,FakeHanoiV2,FakeJakartaV2,FakeJohannesburgV2,FakeKolkataV2,FakeLagosV2,FakeLimaV2,FakeLondonV2,FakeManhattanV2,FakeManilaV2,FakeMelbourneV2,FakeMontrealV2,FakeMumbaiV2,FakeNairobiV2,FakeOslo,FakeOurenseV2,FakeParisV2,FakePerth,FakePrague,FakePoughkeepsieV2,FakeQuitoV2,FakeRochesterV2,FakeRomeV2,FakeSantiagoV2,FakeSherbrooke,FakeSingaporeV2,FakeSydneyV2,FakeTorontoV2,FakeValenciaV2,FakeVigoV2,FakeWashingtonV2,FakeYorktownV2]
#selected_fakes = random.sample(all_fakes, f)
#selected_fakes  # having any randomly selected 5 fake backends out of 46 mentioned in the inital list
selected_fakes=[FakeBoeblingenV2,FakeRomeV2,FakeGeneva,FakeTorontoV2,FakeRochesterV2]

In [24]:
%matplotlib inline
benchmarks=[]
for i in range(lower_limit,upper_limit,steps):
    label = name_of_algo+str(i)
    benchmarks.append((supermarq.vqe_proxy.VQEProxy(i,1),label))
for i in benchmarks:
    #circuit=i[0].qiskit_circuit()
    print("--------------------------",i[1],"---------------------------")
    circuit=i[0].circuit()
    print(circuit)
    #circuit.draw(output='mpl')
    #print(circuit.draw())

-------------------------- vqe4 ---------------------------
[cirq.Circuit([
    cirq.Moment(
        cirq.Ry(rads=3.270861850591521).on(cirq.LineQubit(0)),
        cirq.Ry(rads=3.736922888896891).on(cirq.LineQubit(1)),
        cirq.Ry(rads=4.879243661422773).on(cirq.LineQubit(2)),
        cirq.Ry(rads=7.729070563915058).on(cirq.LineQubit(3)),
    ),
    cirq.Moment(
        cirq.Rz(rads=7.178017644056912).on(cirq.LineQubit(0)),
        cirq.Rz(rads=5.09913840449918).on(cirq.LineQubit(1)),
        cirq.Rz(rads=2.9810989455212966).on(cirq.LineQubit(2)),
        cirq.Rz(rads=9.445286317060221).on(cirq.LineQubit(3)),
    ),
    cirq.Moment(
        cirq.CNOT(cirq.LineQubit(0), cirq.LineQubit(1)),
    ),
    cirq.Moment(
        cirq.CNOT(cirq.LineQubit(1), cirq.LineQubit(2)),
        cirq.Ry(rads=3.731379792847273).on(cirq.LineQubit(0)),
    ),
    cirq.Moment(
        cirq.CNOT(cirq.LineQubit(2), cirq.LineQubit(3)),
        cirq.Ry(rads=6.8125416899714315).on(cirq.LineQubit(1)),
        c

In [25]:
#circuit=i[0].qiskit_circuit()
 assert len(vqe.circuit()) == 2
    assert len(vqe.circuit()[0].all_qubits()) == 3

AttributeError: 'list' object has no attribute 'draw'

In [19]:
def apply_reservoir_gates(qc, n_steps):
    for step in range(n_steps):
        for qubit in range(5):
            qc.rx(np.pi / 4, qubit)  # Apply rotation gate
            qc.ry(np.pi / 4, qubit)  # Apply rotation gate
            qc.rz(np.pi / 4, qubit)  # Apply rotation gate
    return qc

# Create and modify quantum circuit
qc = QuantumCircuit(5)
qc = apply_reservoir_gates(qc, 5)
qc.measure_all()

In [20]:
print(qc)

        ┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐┌─────────┐»
   q_0: ┤ Rx(π/4) ├┤ Ry(π/4) ├┤ Rz(π/4) ├┤ Rx(π/4) ├┤ Ry(π/4) ├┤ Rz(π/4) ├»
        ├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤»
   q_1: ┤ Rx(π/4) ├┤ Ry(π/4) ├┤ Rz(π/4) ├┤ Rx(π/4) ├┤ Ry(π/4) ├┤ Rz(π/4) ├»
        ├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤»
   q_2: ┤ Rx(π/4) ├┤ Ry(π/4) ├┤ Rz(π/4) ├┤ Rx(π/4) ├┤ Ry(π/4) ├┤ Rz(π/4) ├»
        ├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤»
   q_3: ┤ Rx(π/4) ├┤ Ry(π/4) ├┤ Rz(π/4) ├┤ Rx(π/4) ├┤ Ry(π/4) ├┤ Rz(π/4) ├»
        ├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤├─────────┤»
   q_4: ┤ Rx(π/4) ├┤ Ry(π/4) ├┤ Rz(π/4) ├┤ Rx(π/4) ├┤ Ry(π/4) ├┤ Rz(π/4) ├»
        └─────────┘└─────────┘└─────────┘└─────────┘└─────────┘└─────────┘»
meas: 5/══════════════════════════════════════════════════════════════════»
                                                                          »
«        ┌──

In [4]:
benchmark_features = {}
for benchmark, label in benchmarks:
    benchmark_features[label] = [
        supermarq.converters.compute_communication_with_qiskit(benchmark.qiskit_circuit()),
        supermarq.converters.compute_depth_with_qiskit(benchmark.qiskit_circuit()),
        supermarq.converters.compute_entanglement_with_qiskit(benchmark.qiskit_circuit()),
        supermarq.converters.compute_liveness_with_qiskit(benchmark.qiskit_circuit()),
        supermarq.converters.compute_measurement_with_qiskit(benchmark.qiskit_circuit()),
        supermarq.converters.compute_parallelism_with_qiskit(benchmark.qiskit_circuit()),]
print(benchmark_features)

AttributeError: 'list' object has no attribute 'num_qubits'

In [ ]:
import time
import pandas as pd
from IPython.display import display
jobs =[]
data=[]
print("=============SCORES OF ALL THE CIRCUITS=============")    
for backend in selected_fakes:
    print("BACKEND-NAME :: ", backend().name)
    for benchmark, label in benchmarks:
        circuit=benchmark.qiskit_circuit()
        score_list=[]
        timetaken=[]
        for i in range(0,1):
            start=time.time()
            job = backend().run(circuit, shots=1000)
            result = job.result()
            counts = job.result(0).get_counts()
            score = benchmark.score(counts)
            end=time.time()-start
            score_list.append(score)
            t=job.result(0).time_taken
            timetaken.append(t)
            #print(label ," ExecutionTime:",t,"& Total time:",end)
            print(i+1,"th iteration ,", label ,", Execution time::",t, ", Total Time Taken ::",end )
        print("------------------------------------------")
        score = np.mean(score_list) 
        sd = np.std(score_list)
        execution_time = np.mean(timetaken)
        dictionary={}
        dictionary["Backend_Time"]=backend().name
        dictionary["Ciruit_Name"]=label    
        dictionary["Execution_Time"]=execution_time
        dictionary["Total_Time"]=end
        data.append(dictionary)
        jobs.append((label,benchmark,backend,job,score,sd,execution_time,end))
#jobs
df = pd.DataFrame(data)
display(df)
df.to_excel(r"./output_VQE.xlsx", index=False)

In [ ]:

plotdata = pd.DataFrame(df)
plotdata.plot(kind="bar")
plt.title("Mince Pie Consumption Study")
plt.xlabel("Family Member")
plt.ylabel("Pies Consumed")

In [ ]:
df

In [ ]:
all_scores={}  
all_fake_names=[]
all_fake_scores=[]
for i in selected_fakes:
    fakes=i
    #print("SELCETED_FAKES:: ",i)
    i={}
    for label,benchmark,backend,job,score,sd,exec,tot in jobs:
        #supermarq.plotting.plot_results([score], [label])
        if fakes == backend:
            i[label] = score
    device_scores = 'device_score_' +str(fakes().name)
    all_scores[device_scores]= i
    all_fake_scores.append(i)
    all_fake_names.append(device_scores)   
#print(all_scores)    

In [ ]:
for label,benchmark,backend,job,score,sd,exec,tot in jobs:
    print("USED BACKEND::",backend().name,"; CIRCUIT_NAME:",label, "; SCORE =",score)
    supermarq.plotting.plot_results([score], [label])
    print("-----------------------------------------------------------")

In [ ]:
names=[]
scores=[]
X="A "+name_of_algo+' Benchamrk'
for i,j in benchmark_features.items():
    names.append(i)
    scores.append(j)
supermarq.plotting.plot_benchmark([X,names,scores],
        spoke_labels=["PC", "CD", "Ent", "Liv", "Mea", "Par"],show=True)

In [ ]:
supermarq.plotting.plot_correlations(benchmark_features,all_fake_scores,["PC", "CD", "Ent", "Liv", "Mea", "Par"],device_name=all_fake_names,show=True,)

In [ ]:
%matplotlib inline 

all_fake_names=[]
all_score=[]
all_sd=[]
all_ckt=[]
v='Plot of '+name_of_algo+' over'+str(lower_limit)+' to '+str(upper_limit)+ ' Qubits'
for i in selected_fakes:
    all_fake_names.append(i().name)    
for benchmark, label in benchmarks:
    i=[]
    j=[]
    k=[]
    for ckt,benchmark,backend,job,score,sd,exec,tot in jobs:
        if ckt == label:
            i.append(score)
            j.append(sd)
    all_ckt.append(label)
    all_score.append(i)
    all_sd.append(j)
num_bars = len(all_score)
width = 0.1
x = np.arange(len(all_fake_names)) # x-axis positions for each bar
fig, ax = plt.subplots()
for i in range(num_bars):               # Create bars for each category with appropriate offsets
    ax.bar(x + i * width, all_score[i], width=width, label=f'Bar {i+1}', 
           yerr=all_sd[i],align='center',alpha=0.9,ecolor='black',capsize=2)
ax.set_xlabel('Name of Fake Backends',fontweight ='bold',fontsize = 12)
ax.set_ylabel('Scores',fontweight ='bold',fontsize = 12)
ax.set_title(v)
ax.set_xticks(x + (num_bars - 1) * width / 2)
ax.set_xticklabels(all_fake_names)
ax.legend(all_ckt,loc="lower left")
#ax.grid(True)
plt.show()

In [ ]:
import resource
def get_memory():
    usage = resource.getrusage(resource.RUSAGE_SELF)
    max_mem = usage.ru_maxrss/1024 #in MB
    return max_mem
get_memory()